In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
with open("cleaned_pgessay.txt","r", encoding='utf-8') as f:
    text = f.read()

import unicodedata

def normalize_text(text):
    return unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8")

text = normalize_text(text)


#Creating vocab
temp_text = text.replace("<sot>","").replace("<eot>","")
vocab = set(['<sot>','<eot>'])
for ch in temp_text:
    vocab.add(ch)


print(len(vocab), len(text))

72 3240634


In [3]:
chidx = {ch:idx for idx, ch in enumerate(vocab)}
idxch = {idx:ch for ch, idx in chidx.items()}

In [4]:
inputs_x = []
idx=0
while idx<len(text):
    ch = text[idx]
    if text[idx:min(idx+5,len(text))] in vocab:
        inputs_x.append(text[idx:min(idx+5,len(text))])
        idx+=5
    elif text[idx:min(idx+2,len(text))] in vocab:
        inputs_x.append(text[idx:min(idx+2,len(text))])
        idx+=2
    else:
        inputs_x.append(ch)
        idx+=1

In [5]:
seq_length = 25

X = []
y = []

for i in range(len(inputs_x) - seq_length):
    X.append(inputs_x[i:i+seq_length]) 
    y.append(inputs_x[i+seq_length])

In [19]:
vocab_size = len(vocab)
sequnce_length = 25
embedding_dim = 100


model = Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length))
model.add(tf.keras.layers.LSTM(units = 100))
model.add(tf.keras.layers.Dense(vocab_size, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

c:\Users\surya\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [20]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [21]:
X = [[chidx[ele] for ele in x] for x in X]
y = [chidx[ele] for ele in y]

In [22]:
X = np.array(X)
y = np.array(y)

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model.fit(
    X_train, y_train, 
    epochs=15,
    batch_size=64, 
    validation_data=(X_val, y_val),
    verbose=1
)

Epoch 1/15
40485/40485 ━━━━━━━━━━━━━━━━━━━━ 1370s 34ms/step - loss: 1.9415 - val_loss: 1.5286
Epoch 2/15
40485/40485 ━━━━━━━━━━━━━━━━━━━━ 1155s 29ms/step - loss: 1.4978 - val_loss: 1.4495
Epoch 3/15
40485/40485 ━━━━━━━━━━━━━━━━━━━━ 1215s 30ms/step - loss: 1.4363 - val_loss: 1.4213
Epoch 4/15
40485/40485 ━━━━━━━━━━━━━━━━━━━━ 1281s 31ms/step - loss: 1.4073 - val_loss: 1.4039
Epoch 5/15
40485/40485 ━━━━━━━━━━━━━━━━━━━━ 1402s 35ms/step - loss: 1.3914 - val_loss: 1.3929
Epoch 6/15
40485/40485 ━━━━━━━━━━━━━━━━━━━━ 2403s 59ms/step - loss: 1.3787 - val_loss: 1.3816
Epoch 7/15
40485/40485 ━━━━━━━━━━━━━━━━━━━━ 1403s 33ms/step - loss: 1.3689 - val_loss: 1.3754
Epoch 8/15
40485/40485 ━━━━━━━━━━━━━━━━━━━━ 1150s 28ms/step - loss: 1.3646 - val_loss: 1.3711
Epoch 9/15
40485/40485 ━━━━━━━━━━━━━━━━━━━━ 1208s 30ms/step - loss: 1.3580 - val_loss: 1.3683
Epoch 10/15
40485/40485 ━━━━━━━━━━━━━━━━━━━━ 1296s 32ms/step - loss: 1.3513 - val_loss: 1.3657
Epoch 11/15
40485/40485 ━━━━━━━━━━━━━━━━━━━━ 1335s 33ms/ste

In [24]:
def generate_text(seed_text, n_chars_to_generate=100):
    generated_text = seed_text
    
    for _ in range(n_chars_to_generate):
        token_list = [chidx.get(char, 0) for char in generated_text[-seq_length:]]
        token_list = pad_sequences([token_list], maxlen=seq_length, padding='pre')

        predicted_probs = model.predict(token_list, verbose=0)
        # predicted_index = np.argmax(predicted_probs)
        # mixer = {ele:predicted_probs[0][idx] for idx,ele in enumerate(vocab)}
        # print({k: v for k, v in sorted(mixer.items(), key=lambda item: item[1], reverse=True)})
        random_sample = np.random.choice(range(vocab_size), p=predicted_probs.ravel())
        predicted_char = idxch.get(random_sample, '')
        generated_text += predicted_char
    
    return generated_text

In [27]:
generate_text("her fame")

"her fame.you start a startup.with the optimising this professional them that lisp current when they're makin"